# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 观测结果预处理

---
*@author: Evan*\
*@date: 2023-06-06*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.io.shapereader import Reader

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [3]:
# 读取原始数据
xls=pd.read_excel('F:/Data/Project_anqing/May/obsdata/originaldata.xlsx',header=0,na_values=['—',-99])

# 将日期与时间列合并
xls['datetime'] = pd.to_datetime(xls['日期'].astype(str) + ' ' + xls['时间'].astype(str))

# 删除“日期”列和“时间”列
xls.drop(['日期', '时间'], axis=1, inplace=True)

# 设置为索引
xls.set_index('datetime',inplace=True)
xls

,城市,市县,站点,SO2,NO2,NOx,NO,CO,O3,PM10,PM2.5,风速,风向,气压,气温,湿度
datetime,,,,,,,,,,,,,,,,
2023-05-01 00:00:00,安庆市,宜秀区,安庆大学,15.0,30.0,31.0,1.0,0.424,72.0,120.0,35.0,1.0,119.5,1011.2,18.2,54.0
2023-05-01 01:00:00,安庆市,宜秀区,安庆大学,15.0,20.0,21.0,1.0,0.396,74.0,115.0,38.0,1.1,111.8,1010.8,17.6,60.0
2023-05-01 02:00:00,安庆市,宜秀区,安庆大学,10.0,38.0,50.0,8.0,0.432,28.0,113.0,34.0,0.5,302.0,1010.3,14.9,72.0
2023-05-01 03:00:00,安庆市,宜秀区,安庆大学,9.0,54.0,67.0,8.0,0.444,3.0,112.0,34.0,0.2,308.7,1010.2,14.6,76.0
2023-05-01 04:00:00,安庆市,宜秀区,安庆大学,9.0,30.0,32.0,1.0,0.370,32.0,110.0,34.0,0.4,300.9,1010.5,14.6,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31 19:00:00,安庆市,高新区,高新区,NaN,2.0,4.0,2.0,0.475,104.0,22.0,14.0,2.8,296.4,999.3,26.1,81.0
2023-05-31 20:00:00,安庆市,高新区,高新区,NaN,2.0,5.0,2.0,0.523,99.0,22.0,17.0,2.3,293.0,999.6,25.6,84.0
2023-05-31 21:00:00,安庆市,高新区,高新区,NaN,3.0,5.0,1.0,0.506,100.0,40.0,18.0,2.5,291.0,999.8,25.1,85.0


In [4]:
# 将所有因包含括号等字符被读取为字符串的数据设置为空值
xls = xls.replace(to_replace=[r'\(.*?\)'], value=np.nan, regex=True)

In [6]:
xls[xls['站点']=='安庆政务中心']['气压']

datetime
2023-05-01 00:00:00    101.4
2023-05-01 01:00:00    101.3
2023-05-01 02:00:00    101.3
2023-05-01 03:00:00    101.3
2023-05-01 04:00:00    101.3
                       ...  
2023-05-31 19:00:00    100.4
2023-05-31 20:00:00    100.5
2023-05-31 21:00:00    100.5
2023-05-31 22:00:00    100.5
2023-05-31 23:00:00    100.5
Name: 气压, Length: 744, dtype: float64

In [8]:
# 发现安庆政务中心站点的气压数据整体小一个量级
data_replace=xls[xls['站点'].isin(['安庆政务中心'])]['气压']
xls['气压'].replace(data_replace.values,data_replace.values*10,inplace=True)

In [9]:
# 根据站点分组导出
grouped = xls.groupby('站点')
t_index=pd.date_range('2023-05-01T00','2023-05-31T23',freq='h')

for group_name, group_df in grouped:
    # 时间序列如有缺失，则补充空值占位
    group_df.reindex(t_index)
    group_df.to_excel(f'D:/Download/{group_name}.xlsx', index=True)

In [10]:
# 将全部站点取平均并导出
xls.groupby(xls.index).mean().to_excel('D:/Download/allsite.xlsx',index=True)

In [11]:
# 将城区站与通道站分开
urban_path='F:/Data/Project_anqing/May/obsdata/urban/'
channel_path='F:/Data/Project_anqing/May/obsdata/channel/'
other_path='F:/Data/Project_anqing/May/obsdata/other/'
urban_files = [os.path.splitext(filename)[0] for filename in os.listdir(urban_path) if filename.endswith('.xlsx')]
channel_files = [os.path.splitext(filename)[0] for filename in os.listdir(channel_path) if filename.endswith('.xlsx')]
other_files = [os.path.splitext(filename)[0] for filename in os.listdir(other_path) if filename.endswith('.xlsx')]

urban=xls[xls['站点'].isin(urban_files)]
channel=xls[xls['站点'].isin(channel_files)]
other=xls[xls['站点'].isin(other_files)]

# 平均并导出
urban.groupby(urban.index).mean().to_excel('D:/Download/urban.xlsx',index=True)
channel.groupby(channel.index).mean().to_excel('D:/Download/channel.xlsx',index=True)
other.groupby(other.index).mean().to_excel('D:/Download/other.xlsx',index=True)